# IA2A Peru Bank Credit Risk Evaluation Competition

This notebook preprocesses the dataset for the IA2A Peru Bank Credit Risk Evaluation competition,using SQL queries for initial data manipulation, followed by feature importance extraction with XGBoost. Subsequently, it trains a neural network using TensorFlow on the most important features.

Steps:
1. Load and preprocess the data using SQL queries.
2. Use XGBoost to determine feature importance.
3. Select the top N important features.
4. Build and train a neural network using TensorFlow with the selected features.
5. Evaluate the model's performance using the Mean FBeta-Score.
6. Generate predictions for the test dataset and prepare the submission file.


Author: Kevin Juan Román Rafaele

Date: 2024-07-24

License: MIT License

GitHub: https://github.com/PoppinElo/I2A2-Peru-comp

## Import necessary libraries

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import fbeta_score
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

## Load the data

In [46]:
train_path = "/content/I2A2-Peru/train.csv"
test_path = "/content/I2A2-Peru/test_without_labels.csv"
sample_submission_path = "/content/I2A2-Peru/sample_submission.csv"

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
sample_submission_data = pd.read_csv(sample_submission_path)

In [ ]:
train_data.head(2)

,id,status_of_existing_checking_account,duration_in_month,credit_history,purpose,credit_amount,savings_account_or_bounds,present_employment_since,installment_rate_in_percentage_of_disposable_income,personal_status_and_sex,...,property,age_in_years,other_installment_plans,housing,number_of_existing_credits_at_this_bank,job,number_of_people_being_liable_to_provide_maintenance_for,telephone,foreign_worker,good_credit_risk
0,10,0 <= ... < 200 DM,12,existing credits paid back duly till now,car (new),1295,... < 100 DM,... < 1 year,3,female : divorced/separated/married,...,car or other,25,none,rent,1,skilled employee / official,1,none,yes,1
1,82,no checking account,18,existing credits paid back duly till now,business,1568,100 <= ... < 500 DM,1 <= ... < 4 years,3,female : divorced/separated/married,...,building society savings agreement / life insu...,24,none,rent,1,unskilled - resident,1,none,yes,0


In [ ]:
test_data.head(2)

,id,status_of_existing_checking_account,duration_in_month,credit_history,purpose,credit_amount,savings_account_or_bounds,present_employment_since,installment_rate_in_percentage_of_disposable_income,personal_status_and_sex,...,present_residence_since,property,age_in_years,other_installment_plans,housing,number_of_existing_credits_at_this_bank,job,number_of_people_being_liable_to_provide_maintenance_for,telephone,foreign_worker
0,115,no checking account,48,critical account/ other credits existing (not ...,radio/television,3578,unknown/ no savings account,.. >= 7 years,4,male : single,...,1,real estate,47,none,own,1,skilled employee / official,1,yes,yes
1,346,0 <= ... < 200 DM,13,critical account/ other credits existing (not ...,radio/television,882,... < 100 DM,... < 1 year,4,male : single,...,4,real estate,23,none,own,2,skilled employee / official,1,none,yes


**Ensure test_data includes the id column for submission**

In [47]:
test_data_ids = test_data['id']
test_data_ids.head()

0    115
1    346
2    328
3    974
4    587
Name: id, dtype: int64

## Querying with SQL

**Create an SQLite database in memory**

In [ ]:
conn = sqlite3.connect(':memory:')
c = conn.cursor()

**Load data into the database**

In [48]:
train_data.to_sql('train_data', conn, index=False, if_exists='replace')
test_data.to_sql('test_data', conn, index=False, if_exists='replace')

ProgrammingError: Cannot operate on a closed database.

**Example SQL query: Get the average age in years grouped by good credit risk in the training set**

In [ ]:
query = """
SELECT good_credit_risk, AVG(age_in_years) AS avg_age_in_years
FROM train_data
GROUP BY good_credit_risk
"""
avg_age_in_years = pd.read_sql_query(query, conn)
print("\nAverage Age grouped by credit risk:")
print(avg_age_in_years)


Average Age grouped by credit risk:
   good_credit_risk  avg_age_in_years
0                 0         35.891837
1                 1         34.480952


**Example SQL query: Get the types of housing in the training set**

In [ ]:
query = """
SELECT housing AS housing_type, count(housing) AS num_housing
FROM train_data
GROUP BY housing
"""
housing_types = pd.read_sql_query(query, conn)
print("\nHousing Types:")
print(housing_types)


Housing Types:
  housing_type  num_housing
0     for free           75
1          own          502
2         rent          123


**Example SQL query: Get the number of people with good and bad credit risk grouped by sex and by personal status**

In [ ]:
query = """
SELECT
    CASE
        WHEN SUBSTR(personal_status_and_sex, 1, INSTR(personal_status_and_sex, ' ') - 1) = 'female' THEN 'female'
        WHEN SUBSTR(personal_status_and_sex, 1, INSTR(personal_status_and_sex, ' ') - 1) = 'male' THEN 'male'
        ELSE 'unknown'
    END AS sex,
    SUBSTR(personal_status_and_sex, INSTR(personal_status_and_sex, ': ') + 2) AS status,
    good_credit_risk
FROM
    train_data
    LIMIT 10;
"""
good_credit_risk = pd.read_sql_query(query, conn)
print("\nPeople with Good and Bad Credit Risk grouped by Sex and Personal Status:")
print(good_credit_risk)


People with Good and Bad Credit Risk grouped by Sex and Personal Status:
      sex                      status  good_credit_risk
0  female  divorced/separated/married                 1
1  female  divorced/separated/married                 0
2    male                      single                 1
3  female  divorced/separated/married                 0
4    male                      single                 0
5    male                      single                 0
6    male                      single                 0
7    male                      single                 1
8  female  divorced/separated/married                 0
9  female  divorced/separated/married                 0


**Add the new columns in the train_data**

In [49]:

c.execute("""
ALTER TABLE train_data
ADD COLUMN sex TEXT;
""")
c.execute("""
ALTER TABLE train_data
ADD COLUMN status TEXT;
""")

# Commit the changes
conn.commit()

ProgrammingError: Cannot operate on a closed database.

**Update the table to populate the new columns**

In [50]:
c.execute("""
UPDATE train_data
SET
    sex = CASE
        WHEN SUBSTR(personal_status_and_sex, 1, INSTR(personal_status_and_sex, ' ') - 1) = 'female' THEN 'female'
        WHEN SUBSTR(personal_status_and_sex, 1, INSTR(personal_status_and_sex, ' ') - 1) = 'male' THEN 'male'
        ELSE 'unknown'
    END,
    status = SUBSTR(personal_status_and_sex, INSTR(personal_status_and_sex, ': ') + 2);
""")

# Commit the changes
conn.commit()

ProgrammingError: Cannot operate on a closed database.

In [ ]:
query = """
SELECT id, sex, status, personal_status_and_sex
FROM train_data
LIMIT 5
"""
sex_and_status = pd.read_sql_query(query, conn)
print("\nSex and status separated:")
print(sex_and_status)


Sex and status separated:
    id     sex                      status  \
0   10  female  divorced/separated/married   
1   82  female  divorced/separated/married   
2  827    male                      single   
3  410  female  divorced/separated/married   
4   48    male                      single   

               personal_status_and_sex  
0  female : divorced/separated/married  
1  female : divorced/separated/married  
2                        male : single  
3  female : divorced/separated/married  
4                        male : single  


SQLite doesn't support dropping columns directly.
To remove a column, you'll need to create a new table without the column, copy the data, and then rename the new table to the old table's name.

In [ ]:
# Create a new table without the `personal_status_and_sex` column
c.execute("""
CREATE TABLE new_train_table AS
SELECT id, status_of_existing_checking_account, duration_in_month,
       credit_history, purpose, credit_amount,
       savings_account_or_bounds, present_employment_since,
       installment_rate_in_percentage_of_disposable_income,
       sex, status, other_debtors_or_guarantors,
       present_residence_since, property, age_in_years,
       other_installment_plans, housing,
       number_of_existing_credits_at_this_bank, job,
       number_of_people_being_liable_to_provide_maintenance_for,
       telephone, foreign_worker, good_credit_risk
FROM train_data;
""")

# Drop the old table
c.execute("DROP TABLE train_data;")

# Rename the new table to the old table's name
c.execute("ALTER TABLE new_train_table RENAME TO train_data;")

# Commit the changes
conn.commit()

**Create the new train_data version**

In [ ]:
new_train_data = pd.read_sql_query("SELECT * FROM train_data", conn)
new_train_data.head()

ProgrammingError: Cannot operate on a closed database.

**Add the new columns in the test_data**

In [ ]:
c.execute("""
ALTER TABLE test_data
ADD COLUMN sex TEXT;
""")
c.execute("""
ALTER TABLE test_data
ADD COLUMN status TEXT;
""")

# Commit the changes
conn.commit()

ProgrammingError: Cannot operate on a closed database.

**Update the table to populate the new columns in the test_data**

In [ ]:
c.execute("""
UPDATE test_data
SET
    sex = CASE
        WHEN SUBSTR(personal_status_and_sex, 1, INSTR(personal_status_and_sex, ' ') - 1) = 'female' THEN 'female'
        WHEN SUBSTR(personal_status_and_sex, 1, INSTR(personal_status_and_sex, ' ') - 1) = 'male' THEN 'male'
        ELSE 'unknown'
    END,
    status = SUBSTR(personal_status_and_sex, INSTR(personal_status_and_sex, ': ') + 2);
""")

# Commit the changes
conn.commit()

In [ ]:
query = """
SELECT id, sex, status, personal_status_and_sex
FROM test_data
LIMIT 5
"""
sex_and_status = pd.read_sql_query(query, conn)
print("\nSex and status separated:")
print(sex_and_status)


Sex and status separated:
    id     sex                      status  \
0  115    male                      single   
1  346    male                      single   
2  328    male                      single   
3  974  female  divorced/separated/married   
4  587    male                      single   

               personal_status_and_sex  
0                        male : single  
1                        male : single  
2                        male : single  
3  female : divorced/separated/married  
4                        male : single  


SQLite doesn't support dropping columns directly.
To remove a column, you'll need to create a new table without the column,
copy the data, and then rename the new table to the old table's name.



In [51]:
# Create a new table without the `personal_status_and_sex` column
c.execute("""
CREATE TABLE new_test_table AS
SELECT id, status_of_existing_checking_account, duration_in_month,
       credit_history, purpose, credit_amount,
       savings_account_or_bounds, present_employment_since,
       installment_rate_in_percentage_of_disposable_income,
       sex, status, other_debtors_or_guarantors,
       present_residence_since, property, age_in_years,
       other_installment_plans, housing,
       number_of_existing_credits_at_this_bank, job,
       number_of_people_being_liable_to_provide_maintenance_for,
       telephone, foreign_worker
FROM test_data;
""")

# Drop the old table
c.execute("DROP TABLE test_data;")

# Rename the new table to the old table's name
c.execute("ALTER TABLE new_test_table RENAME TO test_data;")

# Commit the changes
conn.commit()

ProgrammingError: Cannot operate on a closed database.

**Create the new test_data version**

In [ ]:
new_test_data = pd.read_sql_query("SELECT * FROM test_data", conn)
new_test_data.head()

,id,status_of_existing_checking_account,duration_in_month,credit_history,purpose,credit_amount,savings_account_or_bounds,present_employment_since,installment_rate_in_percentage_of_disposable_income,sex,...,present_residence_since,property,age_in_years,other_installment_plans,housing,number_of_existing_credits_at_this_bank,job,number_of_people_being_liable_to_provide_maintenance_for,telephone,foreign_worker
0,115,no checking account,48,critical account/ other credits existing (not ...,radio/television,3578,unknown/ no savings account,.. >= 7 years,4,male,...,1,real estate,47,none,own,1,skilled employee / official,1,yes,yes
1,346,0 <= ... < 200 DM,13,critical account/ other credits existing (not ...,radio/television,882,... < 100 DM,... < 1 year,4,male,...,4,real estate,23,none,own,2,skilled employee / official,1,none,yes
2,328,... >= 200 DM / salary assignments for at leas...,36,existing credits paid back duly till now,radio/television,4473,... < 100 DM,.. >= 7 years,4,male,...,2,car or other,31,none,own,1,skilled employee / official,1,none,yes
3,974,no checking account,30,critical account/ other credits existing (not ...,radio/television,2831,... < 100 DM,1 <= ... < 4 years,4,female,...,2,car or other,33,none,own,1,skilled employee / official,1,yes,yes
4,587,... < 0 DM,12,existing credits paid back duly till now,furniture/equipment,1289,... < 100 DM,1 <= ... < 4 years,4,male,...,1,building society savings agreement / life insu...,21,none,own,1,unskilled - resident,1,none,yes


**Close the SQLite connection**

In [ ]:
conn.close()

## Data Preprocessing

**Define data preprocessing function**

In [ ]:
def preprocess_data(data):
    # Handle missing values
    data = data.fillna(-999)  # Simplest way to handle missing values
    # Encode categorical variables
    categorical_features = data.select_dtypes(include=['object']).columns
    for col in categorical_features:
        data[col] = data[col].astype('category').cat.codes
    return data

**Preproccess the new versions**

In [ ]:
train_data = preprocess_data(new_train_data)
test_data = preprocess_data(new_test_data)

**Separate features and target variable**

In [ ]:
X = train_data.drop('good_credit_risk', axis=1)
y = train_data['good_credit_risk']

**Split the data into training and validation sets**

In [52]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

**Feature Scaling**

In [53]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)


# Remove 'id' column from test_data *before* fitting the scaler
test_data_no_id = test_data.drop('id', axis=1)

#Fit the scaler on the training data without the 'id' column
scaler_test = StandardScaler() # Create a new scaler for the test data
X_train_no_id =  train_data.drop('id', axis=1).drop('good_credit_risk', axis=1) # Drop 'id' from the training data
scaler_test.fit(X_train_no_id) #Fit the new scaler to the training data without 'id'
test_data_features = scaler_test.transform(test_data_no_id) # Transform the test data

ValueError: could not convert string to float: '0 <= ... < 200 DM'

**Train an XGBoost model for feature importance**

In [ ]:
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

**Get feature importances and select the top N features**

In [54]:
importances = xgb_model.feature_importances_
indices = np.argsort(importances)[-20:]  # Select top 20 features

**Reduce the feature set to the top N features**

In [55]:
X_train_reduced = X_train[:, indices]
X_val_reduced = X_val[:, indices]
test_data_features_reduced = test_data_features[:, indices-1]

**Define the Mean FBeta-Score function**

In [ ]:
def mean_fbeta_score(y_true, y_pred, beta=1.0):
    return fbeta_score(y_true, y_pred, beta=beta, average='macro')

**Build the model**

In [ ]:
def create_model(input_dim, learning_rate=0.001, dropout_rate=0.5):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

**Initialize the model with the reduced feature set

In [56]:
model = create_model(X_train_reduced.shape[1])

**Set up early stopping and learning rate reduction**

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

**Train the model**

In [ ]:
history = model.fit(
    X_train_reduced, y_train,
    validation_data=(X_val_reduced, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=2
)

Epoch 1/100
18/18 - 3s - loss: 0.7314 - accuracy: 0.5964 - val_loss: 0.6508 - val_accuracy: 0.6571 - lr: 0.0010 - 3s/epoch - 151ms/step
Epoch 2/100
18/18 - 0s - loss: 0.6577 - accuracy: 0.6554 - val_loss: 0.6265 - val_accuracy: 0.6857 - lr: 0.0010 - 128ms/epoch - 7ms/step
Epoch 3/100
18/18 - 0s - loss: 0.5941 - accuracy: 0.6911 - val_loss: 0.6126 - val_accuracy: 0.6786 - lr: 0.0010 - 134ms/epoch - 7ms/step
Epoch 4/100
18/18 - 0s - loss: 0.5542 - accuracy: 0.7357 - val_loss: 0.6003 - val_accuracy: 0.6857 - lr: 0.0010 - 143ms/epoch - 8ms/step
Epoch 5/100
18/18 - 0s - loss: 0.5838 - accuracy: 0.7036 - val_loss: 0.5942 - val_accuracy: 0.7000 - lr: 0.0010 - 201ms/epoch - 11ms/step
Epoch 6/100
18/18 - 0s - loss: 0.5725 - accuracy: 0.7357 - val_loss: 0.5858 - val_accuracy: 0.7000 - lr: 0.0010 - 148ms/epoch - 8ms/step
Epoch 7/100
18/18 - 0s - loss: 0.5589 - accuracy: 0.7196 - val_loss: 0.5835 - val_accuracy: 0.7000 - lr: 0.0010 - 138ms/epoch - 8ms/step
Epoch 8/100
18/18 - 0s - loss: 0.5606 - a

**Evaluate the model**

In [57]:
y_val_pred = (model.predict(X_val_reduced) > 0.5).astype("int32")
fbeta = mean_fbeta_score(y_val, y_val_pred, beta=1.0)
print(f'TensorFlow Model - FBeta-Score: {fbeta:.4f}')

5/5 [==============================] - 0s 3ms/step
TensorFlow Model - FBeta-Score: 0.3507


**Predict on the test set**

In [ ]:
test_predictions = (model.predict(test_data_features_reduced) > 0.5).astype("int32")

10/10 [==============================] - 0s 2ms/step


**Prepare the submission file**

In [58]:
submission = pd.DataFrame({'id': test_data_ids, 'good_credit_risk': test_predictions.flatten()})
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()

,id,good_credit_risk
0,115,0
1,346,0
2,328,1
3,974,0
4,587,0
